# Redesign of nations and how they are created. 

Events are happening all the time. Several events happen before the Character quest begins. 

In [3]:
%load_ext lab_black

import pandas as pd
import numpy as np
import os

import os
import sys
import django

import altair as alt

sys.path.append("../..")
# Loading my project settings from prodweb. This allows me to load and query models.
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "prodweb.settings")
django.setup()

# I'm mapping to the actual files in the repo so that I can also use this to troubleshoot
import sys, pickle

sys.path.append("../")
from lib.builders import people, towns
import lib.drawMaps as draw

In [4]:
with (open("../pickles/billmanhworld.pkl", "rb")) as pickle_file:
    world = pickle.load(pickle_file)

## This will be the new Nations builder module. 

In [42]:
from sklearn.cluster import KMeans


def cluster_nations(world):
    cities = world.df_features[world.df_features["terrain"] == "town"]
    world.nations = []
    world.df_features["nation number"] = np.nan
    world.df_features["nation"] = np.nan
    #     world.df_features["nation number"] = world.df_features["nation number"].
    k = KMeans(init="k-means++", n_clusters=world.culture.n_nations, n_init=10).fit(
        cities[["x", "y"]]
    )
    world.nations_k = k
    predict_nations(world)
    world.nations = [
        Nation(world, cluster=True, k=i) for i in np.unique(world.nations_k.labels_)
    ]
    return k


def predict_nations(world):
    world.df_features["nation number"] = world.nations_k.predict(
        world.df_features[["x", "y"]]
    )


class Nation:
    def __init__(self, world, **kwargs):
        self.name = self.name_nation(world)
        if kwargs.get("cluster", None):
            # Kmeans is the default (when the world is created)
            # Requires the integer value used when creating the world.
            self.cast_nation(world, kwargs.get("k"))
            self.towns = self.addTowns(world)

        self.diplomacy = self.add_diplomacy(world)

    def cast_nation(self, world, k):
        world.df_features.loc[
            world.df_features["nation number"] == k, "nation"
        ] = self.name

    def add_towns(self, world):
        towns = [
            town
            for town in world.towns
            if town.diplomacy.get("nation", "none") == self.name
        ]
        return towns

    def name_nation(self, world):
        nation_name = world.culture.townNameGenerator()
        return nation_name

    def __repr__(self):
        return f"Nation of {self.name}"

    def addTowns(self, world):
        towns = [
            town
            for town in world.towns
            if town.diplomacy.get("nation", "none") == self.name
        ]
        self.towns = towns

    def get_capitol(self):
        c = [t for t in self.towns if t.type == "capitol"][0]
        return c

    def get_random_town(self):
        return np.random.choice(self.towns)

    def getRuler_str(self):
        return self.ruler.name

    def getRuler(self):
        return self.ruler

    def add_diplomacy(self, world):
        # if the world already has nations, then this is a new nation.
        diplomacy = {}
        otherNations = [n for n in world.nations if n.name != self.name]
        for o in otherNations:
            diplomacy[o] = {"favor": 0.8}
        return diplomacy

    def get_deplomacy_df(self):
        d = pd.DataFrame(self.diplomacy).T.reset_index(drop=False)
        d["nation"] = self.name
        d.columns = ["neighbor", "favor", "stance", "nation"]
        return d[["nation", "neighbor", "favor", "stance"]]

    def appointRuler(self, person):
        t = self.getCapitol_str(self.towns)
        self.ruler = person(f"ruler of {self.name}", t)


# nation = Nation(world,k=0)
# print(nation)

In [43]:
k = cluster_nations(world)
k

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=8, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [44]:
k.labels_

array([0, 4, 4, 0, 6, 6, 7, 7, 3, 1, 3, 3, 2, 2, 2, 2, 2, 5, 5],
      dtype=int32)

In [45]:
# [[town for town in nation.towns] for nation in world.nations]

In [46]:
world.df_features.loc[
    world.df_features["terrain"] != "ocean", ["nation number", "nation"]
].groupby("nation").mean().sort_values("nation number")

,nation number
nation,
Tearcatch,0
Godchild,1
Spintear,2
Demonsspring,3
Starcombe,4
Hellpike,5
Malhail,6
Flamecrest,7


In [47]:
[(i, e) for i, e in enumerate(world.nations)]

[(0, Nation of Tearcatch),
 (1, Nation of Godchild),
 (2, Nation of Spintear),
 (3, Nation of Demonsspring),
 (4, Nation of Starcombe),
 (5, Nation of Hellpike),
 (6, Nation of Malhail),
 (7, Nation of Flamecrest)]

In [95]:
len(np.unique(world.df_features["nation"].dropna()))

7

In [98]:
world.df_features[world.df_features["nation"].isna()]

,y,rainfall,x,key,elevation,terrain,feature,nation number,nation,visited,aware,turn_last_visited,danger
key,,,,,,,,,,,,,
0:14,14,0.0,0,0:14,3.0,ocean,NaN,0,NaN,0,1,0,0.122
1:14,14,0.0,1,1:14,1.0,ocean,NaN,0,NaN,0,1,0,0.764
0:15,15,0.0,0,0:15,2.0,ocean,NaN,0,NaN,0,1,0,0.421
1:15,15,0.0,1,1:15,1.0,ocean,NaN,0,NaN,0,1,0,1.081
2:15,15,0.0,2,2:15,2.0,ocean,NaN,0,NaN,0,1,0,0.786
0:16,16,0.0,0,0:16,1.0,ocean,NaN,0,NaN,0,1,0,-0.059
1:16,16,0.0,1,1:16,2.0,ocean,NaN,0,NaN,0,1,0,0.708
2:16,16,0.0,2,2:16,1.0,ocean,NaN,0,NaN,0,1,0,0.686
3:16,16,0.0,3,3:16,4.0,ocean,NaN,0,NaN,0,1,0,1.209


In [99]:
np.unique(world.nations_k.labels_)

array([0, 1, 2, 3, 4, 5, 6, 7], dtype=int32)